# Connecting Notebook to Google Drive

In [3]:
from google.colab import drive

gdrive = drive.mount('/gdrive')

#Project Folder
%cd /gdrive/MyDrive/computer_vision

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/computer_vision


# Seperating Annotations for Training an Testing Data

In [ ]:
# unzip data
!mkdir -p data
!tar -xf train_20241023.tar -C data

In [ ]:
%pip install tensorflow==2.15

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installation

In [ ]:
import os
import cv2

# Paths
data_path = 'data/train_20241023'
daytime_path = os.path.join(data_path, 'daytime')
nighttime_path = os.path.join(data_path, 'nighttime')

# Function to check if label file exists for an image and if it is in the correct format
def check_image_and_label(image_path, label_path):
    # Check if label file exists
    if not os.path.exists(label_path):
        print(f"Warning: Label file missing for image {image_path}.")
        return False

    # Check if image can be read
    img = cv2.imread(image_path)
    if img is None:
        print(f"Warning: Corrupted image {image_path}.")
        return False

    # Check label file format (assuming it's YOLO format: class_id x_center y_center width height)
    try:
        with open(label_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    print(f"Warning: Invalid format in label file {label_path}.")
                    return False
                class_id, x_center, y_center, box_width, box_height = map(float, parts)
                # Ensure the values are within expected ranges
                if not (0 <= class_id and 0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 <= box_width <= 1 and 0 <= box_height <= 1):
                    print(f"Warning: Invalid values in label file {label_path}.")
                    return False
    except Exception as e:
        print(f"Error processing label file {label_path}: {e}")
        return False

    return True

# Function to check the sanity of all images and labels in a folder
def check_data_sanity(folder_path):
    # List all images and labels
    image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]

    for image_filename in image_files:
        image_path = os.path.join(folder_path, image_filename)
        label_path = os.path.join(folder_path, image_filename.replace('.jpg', '.txt'))

        # Check image and label pair
        if not check_image_and_label(image_path, label_path):
            print(f"Sanity check failed for {image_filename}.")
        else:
            print(f"Sanity check passed for {image_filename}.")

# Check sanity of daytime and nighttime folders
print("Checking sanity of daytime data...")
check_data_sanity(daytime_path)

print("\nChecking sanity of nighttime data...")
check_data_sanity(nighttime_path)


In [ ]:
import concurrent.futures
import random
import os
import cv2
import tensorflow as tf

# Paths
data_path = 'data/train_20241023'
daytime_path = os.path.join(data_path, 'daytime')
nighttime_path = os.path.join(data_path, 'nighttime')

# Get image and label files, skipping images with missing label files
def load_images_and_labels(folder_path):
    image_files = []
    label_files = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):  # assuming images are .jpg
            image_path = os.path.join(folder_path, filename)
            label_path = os.path.join(folder_path, filename.replace(".jpg", ".txt"))  # assuming labels are .txt

            if os.path.exists(label_path):
                image_files.append(image_path)
                label_files.append(label_path)
            else:
                print(f"Warning: Label file missing for image {filename}. Skipping this image.")

    return image_files, label_files

day_images, day_labels = load_images_and_labels(daytime_path)
night_images, night_labels = load_images_and_labels(nighttime_path)

# Combine daytime and nighttime images/labels
all_images = day_images + night_images
all_labels = day_labels + night_labels

# Shuffle dataset
combined = list(zip(all_images, all_labels))
random.shuffle(combined)
all_images, all_labels = zip(*combined)

# Convert YOLO format to bounding boxes
def yolo_to_bbox(yolo_label, img_width, img_height):
    with open(yolo_label, "r") as f:
        boxes = []
        for line in f.readlines():
            class_id, x_center, y_center, box_width, box_height = map(float, line.split())
            # Corrected calculation
            x_min = int((x_center - box_width / 2) * img_width)
            y_min = int((y_center - box_height / 2) * img_height)
            x_max = int((x_center + box_width / 2) * img_width)
            y_max = int((y_center + box_height / 2) * img_height)
            boxes.append((class_id, x_min, y_min, x_max, y_max))
    return boxes

# Function to create a TFRecord example
def create_tf_example(image_path, label_path):
    # Load image
    img = cv2.imread(image_path)
    img_height, img_width, _ = img.shape

    # Convert YOLO to bounding box format
    boxes = yolo_to_bbox(label_path, img_width, img_height)

    # Encode image as JPEG
    with tf.io.gfile.GFile(image_path, 'rb') as fid:
        encoded_image_data = fid.read()

    # Create TFRecord example
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[img_height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[img_width])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_image_data])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'jpg'])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=[b[1] / img_width for b in boxes])),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=[b[3] / img_width for b in boxes])),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=[b[2] / img_height for b in boxes])),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=[b[4] / img_height for b in boxes])),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(b[0]) for b in boxes]))
    }))

    return tf_example

# Write TFRecords
def write_tfrecord(output_path, image_list, label_list):
    with tf.io.TFRecordWriter(output_path) as writer:
        for img_path, lbl_path in zip(image_list, label_list):
            tf_example = create_tf_example(img_path, lbl_path)
            writer.write(tf_example.SerializeToString())

# Split dataset into train and test (70:30 ratio)
split_idx = int(0.7 * len(all_images))
train_images, test_images = all_images[:split_idx], all_images[split_idx:]
train_labels, test_labels = all_labels[:split_idx], all_labels[split_idx:]

# Write train and test TFRecords in parallel
write_tfrecord('data/train.tfrecord', train_images, train_labels)
write_tfrecord('data/test.tfrecord', test_images, test_labels)

print("TFRecord files created: train.tfrecord and test.tfrecord")


TFRecord files created: train.tfrecord and test.tfrecord


In [ ]:
def create_label_map(output_path):
    labels = {
        1: "motorbike",
        2: "car",
        3: "van",
        4: "truck"
    }

    with open(output_path, 'w') as f:
        for id, name in labels.items():
            f.write(f"item {{\n  id: {id}\n  name: \"{name}\"\n}}\n\n")

# Specify the output path for the label map file
output_path = 'data/label_map.pbtxt'
create_label_map(output_path)

print(f"Label map file created at: {output_path}")

Label map file created at: data/label_map.pbtxt


In [10]:
# Setting the paths for the records as well as pbtx for easier access later.
train_record_fname = '/gdrive/MyDrive/computer_vision/data/train.tfrecord'
test_record_fname = '/gdrive/MyDrive/computer_vision/data/test.tfrecord'
label_map_pbtxt_fname = '/gdrive/MyDrive/computer_vision/data/label_map.pbtxt'

In [6]:
import tensorflow as tf
raw_dataset = tf.data.TFRecordDataset("/gdrive/MyDrive/computer_vision/data/train.tfrecord")

for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

features {
  feature {
    key: "image/encoded"
    value {
      bytes_list {
        value: "\377\330\377\340\000\020JFIF\000\001\002\000\000\001\000\001\000\000\377\376\000\020Lavc58.35.100\000\377\333\000C\000\01022;2;DDDDDDQKQTTTQQQQTTTZZZjjjZZZTTZZddjjsvsmmjmvv}}}\226\226\217\217\257\257\265\327\327\377\377\304\000\230\000\000\001\005\001\001\001\000\000\000\000\000\000\000\000\000\000\002\005\000\003\004\001\006\007\010\001\000\003\001\001\001\001\000\000\000\000\000\000\000\000\000\000\000\001\002\003\004\005\006\020\000\002\002\000\004\003\005\006\005\004\002\001\004\002\003\001\000\001\002\021\022!\0031AQaq\221\201\"\023\261\004\241\3012\321rRB\3603#\024\341b\361\202S\2224c\242C\322\262\243\223D\021\001\001\001\000\002\002\001\003\003\005\001\001\001\000\000\000\000\001\021\022\0021!AQ\360q\361a\201\003\3412\301\221\241\"\321\377\300\000\021\010\002\320\005\000\003\001\"\000\002\021\000\003\021\000\377\332\000\014\003\001\000\002\021\003\021\000?\000\341e\025\226K\350\207\017

# Install TensorFlow2 Object Detection Dependencies

In [7]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 4305, done.
remote: Counting objects: 100% (4305/4305), done.
remote: Compressing objects: 100% (3292/3292), done.
remote: Total 4305 (delta 1209), reused 2192 (delta 940), pack-reused 0 (from 0)
Receiving objects: 100% (4305/4305), 53.17 MiB | 15.07 MiB/s, done.
Resolving deltas: 100% (1209/1209), done.
Updating files: 100% (3875/3875), done.


# Edit setup.py file for libraries's compatilities
Modify: research/object_detection/packages/tf2/setup.py

In [ ]:
"""Setup script for object_detection with TF2.0."""
import os
from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = [
    # Required for apache-beam with PY3
    'avro-python3',
    'apache-beam',
    'pillow',
    'lxml',
    'matplotlib',
    'Cython',
    'contextlib2',
    'tf-slim',
    'six',
    'pycocotools',
    'lvis',
    'scipy',
    'pandas',
    'tf-models-official >=2.5.1, <2.16.0',
    'tensorflow_io',
    #'keras',
    'pyparsing==2.4.7',  # TODO(b/204103388)
    'sacrebleu<=2.2.0'  # https://github.com/mjpost/sacrebleu/issues/209
]

setup(
    name='object_detection',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    include_package_data=True,
    packages=(
        [p for p in find_packages() if p.startswith('object_detection')] +
        find_packages(where=os.path.join('.', 'slim'))),
    package_dir={
        'datasets': os.path.join('slim', 'datasets'),
        'nets': os.path.join('slim', 'nets'),
        'preprocessing': os.path.join('slim', 'preprocessing'),
        'deployment': os.path.join('slim', 'deployment'),
        'scripts': os.path.join('slim', 'scripts'),
    },
    description='Tensorflow Object Detection Library',
    python_requires='>3.6',
)

In [8]:
# Install the Object Detection API
%%bash
cd ./models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /gdrive/MyDrive/computer_vision/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2024.10.0 requires cloudpickle>=3.0.0, but you have cloudpickle 2.2.1 which is incompatible.


In [1]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

In [4]:
#run model builder test
!python ./models/research/object_detection/builders/model_builder_tf2_test.py

2024-11-12 17:16:33.001992: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-12 17:16:33.002057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 17:16:33.003432: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 17:16:34.036454: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Running tests under Python 3.10.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2024-11-12 17:16:37.024720: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting beca

In [5]:
import tarfile
import shutil
import urllib.request

# Some models to train on ### ssd_mobilenet_v2_coco_2018_03_29
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz',
        'batch_size': 16
    },
}

#in this tutorial we implement the lightweight, smallest state of the art efficientdet model
#if you want to scale up tot larger efficientdet models you will likely need more compute!
chosen_model = 'ssd_mobilenet_v2'

num_steps = 15000 #The more steps, the longer the training. Increase if your loss function is still decreasing and validation metrics are increasing.
num_eval_steps = 500 #Perform evaluation after so many steps

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] #if you can fit a large batch in memory, it may speed up your training

In [6]:
#download pretrained weights
%mkdir ./models/research/deploy/
%cd ./models/research/deploy/
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint

!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

/gdrive/MyDrive/computer_vision/models/research/deploy
--2024-11-12 17:17:13--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.253.118.207, 74.125.200.207, 74.125.130.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|172.253.118.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20518283 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.57M  9.25MB/s    in 2.1s    

2024-11-12 17:17:16 (9.25 MB/s) - ‘ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz’ saved [20518283/20518283]



In [8]:
#download base training configuration file
%cd /gdrive/MyDrive/computer_vision/models/research/deploy
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

/gdrive/MyDrive/computer_vision/models/research/deploy
--2024-11-12 17:18:29--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4684 (4.6K) [text/plain]
Saving to: ‘ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config.1’

ssd_mobilenet_v2_fp 100%[===================>]   4.57K  --.-KB/s    in 0.004s  

2024-11-12 17:18:29 (1.17 MB/s) - ‘ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config.1’ saved [4684/4684]



In [11]:
#prepare
deploy_path = '/gdrive/MyDrive/computer_vision/models/research/deploy/'
pipeline_fname = deploy_path + base_pipeline_file
fine_tune_checkpoint = deploy_path + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)


In [13]:
#write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file

import re

%cd '/gdrive/MyDrive/computer_vision/models/research/deploy/'
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:

    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    f.write(s)


/gdrive/MyDrive/computer_vision/models/research/deploy
writing custom configuration file


In [14]:
%cat /gdrive/MyDrive/computer_vision/models/research/deploy/pipeline_file.config

# SSD with Mobilenet v2 FPN-lite (go/fpn-lite) feature extractor, shared box
# predictor and focal loss (a mobile version of Retinanet).
# Retinanet: see Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from Imagenet classification checkpoint
# Train on TPU-8
#
# Achieves 28.2 mAP on COCO17 Val

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 4
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      multiscale_anchor_generator {

In [15]:
pipeline_file = '/gdrive/MyDrive/computer_vision/models/research/deploy/pipeline_file.config'
model_dir = '/gdrive/MyDrive/computer_vision/training'

In [16]:
import shutil

# Define source and destination file paths
destination_path = '/usr/local/lib/python3.10/dist-packages/tf_slim/data/tfexample_decoder.py'
source_path = '/gdrive/MyDrive/computer_vision/scripts/tfexample_decoder.py'

# Copy the file and overwrite if it exists
shutil.copy2(source_path, destination_path)

print(f"Successfully copied and overwritten {source_path} to {destination_path}")

Successfully copied and overwritten /gdrive/MyDrive/computer_vision/scripts/tfexample_decoder.py to /usr/local/lib/python3.10/dist-packages/tf_slim/data/tfexample_decoder.py


In [17]:
!cat "/usr/local/lib/python3.10/dist-packages/object_detection/data_decoders/tf_example_decoder.py"

# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Tensorflow Example proto decoder for object detection.

A decoder to decode string tensors containing serialized tensorflow.Example
protos for object detection.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import enum
import functools
impo

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

2024-11-12 13:59:30.771306: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-12 13:59:30.771358: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 13:59:30.772995: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 13:59:32.271836: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-11-12 13:59:35.693642: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using Mir